<a href="https://colab.research.google.com/github/Bianca200303/recomendaciones/blob/main/recomendacionvideos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Upload dataset

from google.colab import files
uploaded = files.upload()

Saving GBvideos.csv to GBvideos.csv


In [12]:
import pandas as pd

import string
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
df = pd.read_csv('GBvideos.csv')
print(len(df.index))
print(df.shape)
#explorando etiquetas
print(df['tags'].head)

38916
(38916, 16)
<bound method NDFrame.head of 0        christmas|"john lewis christmas"|"john lewis"|...
1        SNL|"Saturday Night Live"|"SNL Season 43"|"Epi...
2        Eminem|"Walk"|"On"|"Water"|"Aftermath/Shady/In...
3        Salford City FC|"Salford City"|"Salford"|"Clas...
4                                                   [none]
                               ...                        
38911    Enrique Iglesias feat. Pitbull|"MOVE TO MIAMI"...
38912    jacob sartorius|"jacob"|"up with it"|"jacob sa...
38913    anne|"marie"|"anne-marie"|"2002"|"two thousand...
38914    Eurovision Song Contest|"2018"|"Lisbon"|"Cypru...
38915    Kyle|"SuperDuperKyle"|"Ikuyo"|"2 Chainz"|"Soph...
Name: tags, Length: 38916, dtype: object>


In [ ]:
#limpiamos la estructura de los datos

In [11]:
df['ntags'] = df['tags'].str.replace('[{}]'.format(string.punctuation), ' ', regex=True)
print(df['ntags'].head())


0    christmas  john lewis christmas   john lewis  ...
1    SNL  Saturday Night Live   SNL Season 43   Epi...
2    Eminem  Walk   On   Water   Aftermath Shady In...
3    Salford City FC  Salford City   Salford   Clas...
4                                                none 
Name: ntags, dtype: object


In [15]:
#excluimos abras que no agregan valor
tfidf=TfidfVectorizer(stop_words='english')
#rellenar espacios sin datos con ''
df['ntags']=df['ntags'].fillna('')


0    christmas  john lewis christmas   john lewis  ...
1    SNL  Saturday Night Live   SNL Season 43   Epi...
2    Eminem  Walk   On   Water   Aftermath Shady In...
3    Salford City FC  Salford City   Salford   Clas...
4                                                none 
Name: ntags, dtype: object


In [19]:
tfidf_matrix=tfidf.fit_transform(df['ntags'])
print(tfidf.vocabulary_)
print(len(tfidf.vocabulary_))
print(tfidf_matrix.shape)

{'christmas': 2611, 'john': 6944, 'lewis': 7735, 'ad': 395, 'mozthemonster': 9010, '2017': 138, 'advert': 439, 'moz': 9009, 'snl': 12265, 'saturday': 11577, 'night': 9368, 'live': 7867, 'season': 11722, '43': 230, 'episode': 4421, '1730': 76, 'tiffany': 13339, 'haddish': 5804, 'taylor': 13104, 'swift': 12949, 'ready': 10881, 's43': 11443, 's43e5': 11456, 'new': 9310, 'york': 14771, 'comedy': 2860, 'sketch': 12138, 'funny': 5194, 'hilarious': 6131, 'late': 7561, 'host': 6285, 'music': 9096, 'guest': 5736, 'laugh': 7573, 'impersonation': 6506, 'actor': 390, 'improv': 6518, 'musician': 9102, 'comedian': 2857, 'actress': 392, 'loving': 8011, 'wrong': 14635, 'oprah': 9683, 'winfrey': 14499, 'girls': 5452, 'trip': 13634, 'carmichael': 2253, 'keanu': 7174, 'reputation': 11051, 'look': 7946, 'eminem': 4322, 'walk': 14240, 'water': 14311, 'aftermath': 459, 'shady': 11866, 'interscope': 6622, 'rap': 10820, 'salford': 11507, 'city': 2666, 'fc': 4742, 'class': 2693, '92': 311, 'university': 13882,